In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [5]:
df = pd.read_csv('df_adjusted_date.csv')

In [6]:
# Change days to using adjusted date
df['adjusted_date'] = df['adjusted_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df['DAY'] = df['adjusted_date'].dt.day
df['MONTH'] = df['adjusted_date'].dt.month
df['YEAR'] = df['adjusted_date'].dt.year

In [7]:
# insert '0' and convert to string
df['DAY'] = df['DAY'].apply(lambda x: int(float(x)))
df['DAY'] = df['DAY'].apply('{0:0>2}'.format)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)

In [8]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,time=time,method='nearest') 
    cloud_fraction = dsloc['hotspot']
    return cloud_fraction 

In [9]:
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat,
        lon = row.lon
        time = row.adjusted_date
        path = home / Path("D:/HS_1985-2018_consecsum_gtc0.nc") 
        array = file_info(path)
        cf = array_sel(array)
        x.append(cf)
sst = [item[0].values for item in x]
df['SST'] = sst
df.to_csv('HSadjusteddate.csv')
